In [53]:
"""This file contains code for use with "Think Stats",
by Allen B. Downey, available from greenteapress.com

Copyright 2010 Allen B. Downey
License: GNU GPLv3 http://www.gnu.org/licenses/gpl.html
"""

import sys
import numpy as np
import thinkstats2
import first
import math

from collections import defaultdict


def ReadFemResp(dct_file='2002FemResp.dct',
                dat_file='2002FemResp.dat.gz',
                nrows=None):
    """Reads the NSFG respondent data.

    dct_file: string file name
    dat_file: string file name

    returns: DataFrame
    """
    dct = thinkstats2.ReadStataDct(dct_file)
    df = dct.ReadFixedWidth(dat_file, compression='gzip', nrows=nrows)
    CleanFemResp(df)
    return df


def CleanFemResp(df):
    """Recodes variables from the respondent frame.

    df: DataFrame
    """
    pass


def ReadFemPreg(dct_file='2002FemPreg.dct',
                dat_file='2002FemPreg.dat.gz'):
    """Reads the NSFG pregnancy data.

    dct_file: string file name
    dat_file: string file name

    returns: DataFrame
    """
    dct = thinkstats2.ReadStataDct(dct_file)
    df = dct.ReadFixedWidth(dat_file, compression='gzip')
    CleanFemPreg(df)
    return df


def CleanFemPreg(df):
    """Recodes variables from the pregnancy frame.

    df: DataFrame
    """
    # mother's age is encoded in centiyears; convert to years
    df.agepreg /= 100.0

    # birthwgt_lb contains at least one bogus value (51 lbs)
    # replace with NaN
    df.loc[df.birthwgt_lb > 20, 'birthwgt_lb'] = np.nan
    
    # replace 'not ascertained', 'refused', 'don't know' with NaN
    na_vals = [97, 98, 99]
    df.birthwgt_lb.replace(na_vals, np.nan, inplace=True)
    df.birthwgt_oz.replace(na_vals, np.nan, inplace=True)
    df.hpagelb.replace(na_vals, np.nan, inplace=True)

    df.babysex.replace([7, 9], np.nan, inplace=True)
    df.nbrnaliv.replace([9], np.nan, inplace=True)

    # birthweight is stored in two columns, lbs and oz.
    # convert to a single column in lb
    # NOTE: creating a new column requires dictionary syntax,
    # not attribute assignment (like df.totalwgt_lb)
    df['totalwgt_lb'] = df.birthwgt_lb + df.birthwgt_oz / 16.0    

    # due to a bug in ReadStataDct, the last variable gets clipped;
    # so for now set it to NaN
    df.cmintvw = np.nan


def ValidatePregnum(resp, preg):
    """Validate pregnum in the respondent file.

    resp: respondent DataFrame
    preg: pregnancy DataFrame
    """
    # make the map from caseid to list of pregnancy indices
    preg_map = MakePregMap(preg)
    
    # iterate through the respondent pregnum series
    for index, pregnum in resp.pregnum.iteritems():
        caseid = resp.caseid[index]
        indices = preg_map[caseid]

        # check that pregnum from the respondent file equals
        # the number of records in the pregnancy file
        if len(indices) != pregnum:
            print(caseid, len(indices), pregnum)
            return False

    return True


def MakePregMap(df):
    """Make a map from caseid to list of preg indices.

    df: DataFrame

    returns: dict that maps from caseid to list of indices into `preg`
    """
    d = defaultdict(list)
    for index, caseid in df.caseid.iteritems():
        d[caseid].append(index)
    return d


def main():
    """Tests the functions in this module.

    script: string script name
    """
    # read and validate the respondent file
    resp = ReadFemResp()

    assert(len(resp) == 7643)
    assert(resp.pregnum.value_counts()[1] == 1267)

    # read and validate the pregnancy file
    preg = ReadFemPreg()
    print(preg.shape)

    assert len(preg) == 13593
    assert preg.caseid[13592] == 12571
    assert preg.pregordr.value_counts()[1] == 5033
    assert preg.nbrnaliv.value_counts()[1] == 8981
    assert preg.babysex.value_counts()[1] == 4641
    assert preg.birthwgt_lb.value_counts()[7] == 3049
    assert preg.birthwgt_oz.value_counts()[0] == 1037
    assert preg.prglngth.value_counts()[39] == 4744
    assert preg.outcome.value_counts()[1] == 9148
    assert preg.birthord.value_counts()[1] == 4413
    assert preg.agepreg.value_counts()[22.75] == 100
    assert preg.totalwgt_lb.value_counts()[7.5] == 302

    weights = preg.finalwgt.value_counts()
    key = max(weights.keys())
    assert preg.finalwgt.value_counts()[key] == 6

    # validate that the pregnum column in `resp` matches the number
    # of entries in `preg`
    assert(ValidatePregnum(resp, preg))

    
    print('All tests passed.')


if __name__ == '__main__':
    main()

(13593, 244)
All tests passed.


C:\Users\brean\AppData\Local\Temp\ipykernel_38000\1632307760.py:119: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for index, caseid in df.caseid.iteritems():
C:\Users\brean\AppData\Local\Temp\ipykernel_38000\1632307760.py:98: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for index, pregnum in resp.pregnum.iteritems():


Select the birthord column, print the value counts, and compare to results published in the codebook

In [ ]:
#assigned function to variable to pull data

In [21]:
preg = ReadFemPreg()
print(preg.birthord.value_counts())

1.0     4413
2.0     2874
3.0     1234
4.0      421
5.0      126
6.0       50
7.0       20
8.0        7
9.0        2
10.0       1
Name: birthord, dtype: int64


We can also use isnull to count the number of nans.

In [23]:
print(preg.birthord.isnull().sum())

4445


Select the prglngth column, print the value counts, and compare to results published in the codebook

In [24]:
print(preg.prglngth.value_counts().sort_index())

0       15
1        9
2       78
3      151
4      412
5      181
6      543
7      175
8      409
9      594
10     137
11     202
12     170
13     446
14      29
15      39
16      44
17     253
18      17
19      34
20      18
21      37
22     147
23      12
24      31
25      15
26     117
27       8
28      38
29      23
30     198
31      29
32     122
33      50
34      60
35     357
36     329
37     457
38     609
39    4744
40    1120
41     591
42     328
43     148
44      46
45      10
46       1
47       1
48       7
50       2
Name: prglngth, dtype: int64


To compute the mean of a column, you can invoke the mean method on a Series. For example, here is the mean birthweight in pounds:

In [25]:
print(preg.totalwgt_lb.mean())

7.265628457623368


Create a new column named totalwgt_kg that contains birth weight in kilograms.Compute its mean. Remember that when you create a new column, you have to use dictionary syntax, not dot notation

In [ ]:
#converting lb to kg

In [26]:
preg['totalwgt_kg'] = preg.totalwgt_lb /2.2
print(preg.totalwgt_kg.mean())

3.302558389828803


nsfg.py also provides ReadFemResp, which reads the female respondents file and returns a DataFrame:

In [ ]:
#assigning another variable to a function above to pull data from

In [29]:
resp = ReadFemResp()

DataFrame provides a method head that displays the first five rows:

In [31]:
print(resp.head())

   caseid  rscrinf  rdormres  rostscrn  rscreenhisp  rscreenrace  age_a  \
0    2298        1         5         5            1          5.0     27   
1    5012        1         5         1            5          5.0     42   
2   11586        1         5         1            5          5.0     43   
3    6794        5         5         4            1          5.0     15   
4     616        1         5         4            1          5.0     20   

   age_r  cmbirth  agescrn  ...  pubassis_i      basewgt  adj_mod_basewgt  \
0     27      902       27  ...           0  3247.916977      5123.759559   
1     42      718       42  ...           0  2335.279149      2846.799490   
2     43      708       43  ...           0  2335.279149      2846.799490   
3     15     1042       15  ...           0  3783.152221      5071.464231   
4     20      991       20  ...           0  5341.329968      6437.335772   

      finalwgt  secu_r  sest  cmintvw  cmlstyr  screentime   intvlngth  
0  5556.71724

Select the age_r column from resp and print the value counts. How old are the youngest and oldest respondents?

In [ ]:
#youngest is 15 while oldest is 44. 

In [32]:
print(resp.age_r.value_counts().sort_index())

15    217
16    223
17    234
18    235
19    241
20    258
21    267
22    287
23    282
24    269
25    267
26    260
27    255
28    252
29    262
30    292
31    278
32    273
33    257
34    255
35    262
36    266
37    271
38    256
39    215
40    256
41    250
42    215
43    253
44    235
Name: age_r, dtype: int64


We can use the caseid to match up rows from resp and preg. For example, we can select the row from resp for caseid 2298 like this:

In [33]:
print(resp[resp.caseid==2298])

   caseid  rscrinf  rdormres  rostscrn  rscreenhisp  rscreenrace  age_a  \
0    2298        1         5         5            1          5.0     27   

   age_r  cmbirth  agescrn  ...  pubassis_i      basewgt  adj_mod_basewgt  \
0     27      902       27  ...           0  3247.916977      5123.759559   

      finalwgt  secu_r  sest  cmintvw  cmlstyr  screentime   intvlngth  
0  5556.717241       2    18     1234     1222    18:26:36  110.492667  

[1 rows x 3087 columns]


And we can get the corresponding rows from preg like this:

In [34]:
print(preg[preg.caseid==2298])

      caseid  pregordr  howpreg_n  howpreg_p  moscurrp  nowprgdk  pregend1  \
2610    2298         1        NaN        NaN       NaN       NaN       6.0   
2611    2298         2        NaN        NaN       NaN       NaN       6.0   
2612    2298         3        NaN        NaN       NaN       NaN       6.0   
2613    2298         4        NaN        NaN       NaN       NaN       6.0   

      pregend2  nbrnaliv  multbrth  ...  religion_i  metro_i      basewgt  \
2610       NaN       1.0       NaN  ...           0        0  3247.916977   
2611       NaN       1.0       NaN  ...           0        0  3247.916977   
2612       NaN       1.0       NaN  ...           0        0  3247.916977   
2613       NaN       1.0       NaN  ...           0        0  3247.916977   

      adj_mod_basewgt     finalwgt  secu_p  sest  cmintvw  totalwgt_lb  \
2610      5123.759559  5556.717241       2    18      NaN       6.8750   
2611      5123.759559  5556.717241       2    18      NaN       5.5000   
2

How old is the respondent with caseid 1?

In [35]:
print(resp[resp.caseid==1].age_r)

1069    44
Name: age_r, dtype: int64


What are the pregnancy lengths for the respondent with caseid 2298?

In [36]:
print(preg[preg.caseid==2298].prglngth)

2610    40
2611    36
2612    30
2613    40
Name: prglngth, dtype: int64


What was the birthweight of the first baby born to the respondent with caseid 5012?

In [38]:
print(preg[preg.caseid==5012].birthwgt_lb)

5515    6.0
Name: birthwgt_lb, dtype: float64


Page 25: 2-1 (Based on the results in this chapter, suppose you were asked to summarize what you learned about 
              #whether first babies arrive late…)

in the first histogram in the chapter comparing first vs other pregnancies, first pregnancies do have a higher frequency of being born later in term than other pregnancies. The author states that there is a skew because there are more other births over first births so we have a smaller sample size of first births. We could possibly fix this by getting data from mothers who only had one child and see how long their pregnancy lasted and add that to our data set. Or we could create a data set only with mothers with two children and create a histogram on the first vs second bith lengths to compare those. It is clear that more data is needed to determine if this is a true statement. However, with the data given, first births have a greater frequency of being born past 40 weeks over other births. If we look at the mean and Cohen's d for this data though, we can see that this difference is very small and not noticable at all.   

Page 25: 2-4 (Using the variable totalwgt_lb, investigate whether first babies are lighter or heavier than others…)

In [ ]:
#first we want to only include live births. then we split this up based on whether it was a first birth or not. 
#we caclulate the difference between the mean of the two, along with the variance and Cohen's d

In [58]:
# def CohenEffectSize(firsts, others):
live = preg[preg.outcome == 1]
firsts = live[live.birthord == 1]
others = live[live.birthord != 1]
diff = firsts.totalwgt_lb.mean() - others.totalwgt_lb.mean()
print("first babies birth weight mean", firsts.totalwgt_lb.mean()) 
print("other babies first weight mean", others.totalwgt_lb.mean())
var1 = firsts.totalwgt_lb.var()
var2 = others.totalwgt_lb.var()
n1,n2 = len(firsts), len(others)
    
pooled_var = (n1 * var1 + n2 * var2) / (n1 + n2)
d = diff / math.sqrt(pooled_var)
print("cohen's d", d)

first babies birth weight mean 7.201094430437772
other babies first weight mean 7.325855614973262
cohen's d -0.08867292707260174


#according to the above data, first babies are slightly lighter than other babies born. However, since Cohen's D is less than .1, we can assume that this difference in weight is negligible. 